In [ ]:
def listAsFloats(congList):
    anArray = []
    congList2 = congList.copy()
    for congress in congList2:
        congress = congress.pivot(index = 'icpsr', columns = 'rollnumber')
        gowerVotes.append(congress)
        congress = congress.reset_index()
#     congress['icpsr'].astype(int)
        anArray.append((congress.values.tolist()))
    #for vote in finalVotes:
      #  for each in vote:
      #      index= 1
      #      length = len(each)
      #      while(index < length):
       #         if each[index] == '2':
       #             each[index] = 'None'
       #         index +=1
    return anArray

In [ ]:
import pandas as pd
votes = pd.read_csv('Hall_votes.csv')
votes = votes.drop(columns = ['chamber', 'prob'])
castCode = []
for line in votes.cast_code:
    if line >= 7:
        castCode.append(-1)
    elif line > 3:
        castCode.append(1)
    else:
        castCode.append(0)
votes.cast_code = castCode
congNum = 1
congList = []
while(congNum < 117):
    congList.append((votes[votes['congress'] == congNum]).drop(columns = 'congress'))
    congNum += 1
finalVotes = []
gowerVotes = []
congFloats = listAsFloats(congList)
for congress in congList:
    congress = congress.pivot(index = 'icpsr', columns = 'rollnumber')
    #gowerVotes.append(congress)
    congress = congress.reset_index()
#     congress['icpsr'].astype(int)
    finalVotes.append((congress.values.astype(str).tolist()))
for vote in finalVotes:
    for each in vote:
        index= 1
        length = len(each)
        while(index < length):
            if each[index] == '2':
                each[index] = 'None'
            index +=1

In [ ]:
import gower
t = gowerVotes[0]
# print(t)
features = [True for x in range(0, len(t.columns))]
distance_matrix = gower.gower_matrix(data_x=t, cat_features=features)
dmdf = pd.DataFrame(distance_matrix)
#dmdf

In [ ]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters=2, affinity='precomputed', linkage='complete')
clustering.fit_predict(distance_matrix)

In [ ]:
party = pd.read_csv('Hall_members.csv')
party = party.drop(columns = ['chamber', 'state_icpsr', 'district_code', 'state_abbrev', 'occupancy', 'last_means', 'bioname',
                             'bioguide_id', 'born', 'died','conditional','nokken_poole_dim1', 'nokken_poole_dim2', 
                            'nominate_dim1', 'nominate_dim2', 'nominate_number_of_errors', 'nominate_log_likelihood','nominate_geo_mean_probability',
                             'nominate_number_of_votes'])

party = party.set_index('congress')

congList = []
congNum = 1
while(congNum < 117):
    newCong = party.loc[congNum]
    newCong = newCong.sort_values(by=['icpsr'])
    newCong = newCong.reset_index(drop=True)
    congList.append(newCong)
    congNum += 1
for cong in congList:
    newPartyList= []
    parties = cong['party_code'].value_counts().index.tolist()
    for each in cong['party_code']:
        if each == parties[0]:
            newPartyList.append(0)
        elif each == parties[1]:
            newPartyList.append(1)
        else:
            newPartyList.append(-1)
    cong.party_code = newPartyList

finalParty = []
for cong in congList:
    finalParty.append(cong.values.tolist())

In [ ]:
def dropMissing(congVote, congParty):
    LVote = len(congVote)
    PVote = len(congParty)
    if(LVote == PVote):
        return congVote, congParty
    print(LVote)
    print(PVote)
    maxC = LVote if LVote > PVote else PVote
    z = 0
    while(z < maxC):
        if float(congVote[z][0]) != congParty[z][0]:
            if LVote < PVote:
                print("dropping")
                print(congParty[z])
                del congParty[z]
                z-=1
                maxC -= 1
            elif PVoe > LVote:
                print("dropping")
                print(congVote[z])
                del congVote[z]
                z-=1
                macC -= 1
            else:
                print("Same length different values")
        z+=1
    return congVote, congParty

In [ ]:
import copy
def getVote(num):
    votes = copy.deepcopy(finalVotes)
    return votes[num - 1]

In [ ]:
def getParty(num):
    partys = finalParty[:]
    return partys[num - 1]

In [ ]:
def checkAccuracy(trueParty, testParty):
    index = 0
    count = len(trueParty)
    trueCount = 0
    while(index < count):
        if trueParty[index] == testParty[index]:
            trueCount += 1
        index += 1
    accuracy = trueCount/count
    return accuracy

In [ ]:
def getClusters(x):
    kmodes_cao = KModes(n_clusters=2, init='Cao', verbose=1)
    kmodes_cao.fit_predict(x)

#     Print cluster centroids of the trained model.
#     print('k-modes (Cao) centroids:')
#     print(kmodes_cao.cluster_centroids_)
    
#     Print training statistics
#     print('Final training cost: {}'.format(kmodes_cao.cost_))
#     print('Training iterations: {}'.format(kmodes_cao.n_iter_))
#     print(kmodes_cao.labels_)
#     print(len(kmodes_cao.labels_))
    return kmodes_cao

In [ ]:
def getAgglomerativeClusters(x):
    features = [True for x in range(0, len(x.columns))]
    print(x)
    gower_distances = gower.gower_matrix(data_x=x, cat_features=features)
    aggclustering = AgglomerativeClustering(n_clusters=2, affinity='precomputed', linkage='complete')
    return aggclustering.fit_predict(gower_distances)

In [ ]:
def testClusters(icpsrList, clustMap):
    
    clustMapCopy = copy.deepcopy(clustMap)
    cLen = len(clustMapCopy)
    ind = 0
    while(ind < cLen):
        if clustMapCopy[ind] == 0:
            clustMapCopy[ind] = 1
        else:
            clustMapCopy[ind] = 0
        ind+=1
    index = 0
    testParty = [] #first clust
    testParty2 = []
    while(index < len(icpsrList)):
        mem = icpsrList[index]
        c = clustMap[index]
        d = clustMapCopy[index]
        testParty.append([mem, c])
        testParty2.append([mem, d])
        index +=1
    return testParty, testParty2

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import gower
import numpy as np
import difflib

loop =  116 #len(finalVotes)
i = 1
accuracy_array = list(range(0, 115))
while(i < loop):
    
    print("Congress Being Tested: ")
    print(i)
    
    aParty = getParty(i)
    aVote = getVote(i)
    aVote, aParty = dropMissing(aVote, aParty)
    
    icpsrList = []
    for each in aVote:
        icpsrList.append(float(each.pop(0))) #seperate icpsr from votes cast into icpsrList and x
    x = np.array(aVote)
    #print(x)
    #print(pd.DataFrame(congFloats[0]))
    floatData = pd.DataFrame(congFloats[(i-1)])
    formattedData = floatData.drop(floatData.columns[[0]], axis=1)
    #print(formattedData)
    aggclusters = getAgglomerativeClusters(formattedData)
    #clusters = kmodesCoa.cluster_centroids_
    clustMap = aggclusters
    print(clustMap)
    #print(len(clustMap))
    #print(len(icpsrList))
    testParty, testParty2 = testClusters(icpsrList, clustMap)
    acc = checkAccuracy(aParty, testParty)
    acc2 = checkAccuracy(aParty, testParty2)
    #print(acc if acc > acc2 else acc2)
    highest_acc = acc if acc > acc2 else acc2
    #print(i)
    accuracy_array[(i-1)] = highest_acc
    print(highest_acc)
#     print(aParty)
#     print(testParty if acc > acc2 else testParty2)
    i+=1
avg_acc = (sum(accuracy_array))/(len(accuracy_array))
print("Average accuracy:" + str(avg_acc))

In [ ]:
avg_acc = (sum(accuracy_array))/(len(accuracy_array))
print("Average accuracy:" + str(avg_acc))

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics


#takes a distance matrix, the epsilon size, and core group size
#returns a DBSCAN object
def tryDBScan(dMatrix, eps_size, core_size):

    return DBSCAN(eps = eps_size, min_samples = core_size, metric=“precomputed”).fit(dMatrix)

n=.7
while(n>.00001):
    db = tryDBScan(dMatrix, n, 5)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
   
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
   
    if(n_clusters <= 1):
        n=n/2
    else:
        print("Found %n clusters with %f eps" %n_clusters %n)
        break